In [1]:
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from skimage import data
from skimage import filters, io
from skimage.filters import try_all_threshold
from scipy import ndimage as ndi
from scipy import stats
import numpy as np
import glob

In [2]:
from tqdm import tqdm
x_data = []
y_data = []
Fonts = [ 'IBM Plex Sans Arabic', 'Lemonada', 'Marhey','Scheherazade New'] 

for font in Fonts:
    filenames = sorted(glob.glob(f'./fonts-dataset/{font}/*.jpeg'))
    for filename in tqdm(filenames, desc=f"Processing {font}"):
       img =ski.io.imread(filename)   
       filtered_img = ski.filters.median(img,mode='nearest')
       grayImg = ski.color.rgb2gray(filtered_img)
       thresh= ski.filters.threshold_otsu(grayImg)#neb2a nshof meen
       binary = grayImg < thresh
       binaryImg = img = ski.util.img_as_ubyte(binary)
       black_pixels = np.sum(binaryImg == 0)
       total_pixels = binaryImg.size
       percentage_black = (black_pixels / total_pixels) * 100
       if percentage_black > 50:
           inverted_binaryImg = np.where(binaryImg == 0, 255, 0)
       else:
           inverted_binaryImg = binaryImg
       inverted_binaryImg = ski.util.img_as_ubyte(inverted_binaryImg)
       edges = ski.feature.canny(img, sigma=4)
       h, theta, d = ski.transform.hough_line(edges)
       accum, angles, dists = ski.transform.hough_line_peaks(h, theta, d)
       # Calculate the mode of the angles
       if len(angles) > 0:
           amgles_deg = np.rad2deg(angles)+90
           angl_mode = stats.mode(amgles_deg)
           rotation_angle = angl_mode.mode
       
           # Rotate the image
           rotated_img = ski.transform.rotate(inverted_binaryImg, rotation_angle, cval=1,resize=False)
           rotated_img = ski.util.img_as_ubyte(rotated_img)
       else:
           # Skip the rotation if no angles were found
           rotated_img = inverted_binaryImg
       x_data.append(rotated_img)
       
       
x_data = np.asarray(x_data)
y_data = np.asarray(y_data)


print("shape of x_data:", x_data.shape, "shape of y_data:", y_data.shape)

Processing IBM Plex Sans Arabic:   0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\Momo\AppData\Local\Programs\Python\Python311\Lib\site-packages\skimage\util\dtype.py:576: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  return _convert(image, np.uint8, force_copy)
Processing Scheherazade New: 100%|██████████| 1000/1000 [13:15<00:00,  1.26it/s]


shape of x_data: (4000, 1181, 1181) shape of y_data: (0,)


In [9]:
font_to_num = {font: i for i, font in enumerate(Fonts)}

y_data = []
for font in Fonts:
    filenames = sorted(glob.glob(f'./fonts-dataset/{font}/*.jpeg'))
    for filename in tqdm(filenames, desc=f"Processing {font}"):
        y_data.append(font_to_num[font])

y_data = np.asarray(y_data)

Processing Lemonada: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


Processing Scheherazade New: 100%|██████████| 1000/1000 [00:00<00:00, 545636.01it/s]


In [10]:
import cv2
import os
import numpy as np

# Assuming preprocessed_images is a list of your preprocessed images
preprocessed_images = x_data

# Specify the directory you want to save the images in
save_dir = "preprocessed_images"

# Make sure the directory exists, if not, create it
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Loop through your preprocessed images
for i, img in enumerate(preprocessed_images):
    # Convert the image to uint8
    img_uint8 = (img).astype(np.uint8)
    
    # Define the name of the file
    filename = os.path.join(save_dir, f"{y_data[i]}_{i}.jpg")
    
    # Save the image
    cv2.imwrite(filename, img_uint8)

In [17]:
import cv2
import os
import numpy as np

# Specify the directory where your preprocessed images are saved
img_dir = "preprocessed_images"

# Get the list of image file names
img_files = os.listdir(img_dir)

# Initialize an empty list to store the images
x_data = []
y_data = []

# Loop through the image files
for img_file in img_files:
    # Construct the full path of the image file
    img_path = os.path.join(img_dir, img_file)
    
    # Read the image using cv2.imread
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Use cv2.IMREAD_COLOR for color images
    
    # Append the image to x_data
    x_data.append(img)
    label=img_file.split('_')[0]
    y_data.append(label)

# Convert x_data to a numpy array
x_data = np.array(x_data)
y_data = np.array(y_data) 


# Now you can use x_data in your code
p = np.random.permutation(len(x_data))
x_data = x_data[p]
y_data = y_data[p]
x_images = x_data
del img_files


In [18]:
from tqdm import tqdm
from cv2 import resize
import skimage as ski
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import svm

hog_features = []

batch_size = 100  # Adjust this based on your available memory
num_batches = len(x_data) // batch_size

for i in tqdm(range(num_batches + 1), desc="Processing batches"):  # +1 to include the last batch
    start = i * batch_size
    end = min(start + batch_size, len(x_data))  # Ensure the end index doesn't exceed the number of images
    batch = x_data[start:end]
    
    for image in batch:
        # Resize the image
        image = resize(image, (1000, 1000))
        
        # Apply HOG feature extraction
        hog_feat = ski.feature.hog(image, orientations=15, pixels_per_cell=(70,55), cells_per_block=(14, 11), block_norm='L2')
        hog_features.append(hog_feat)

scaler = StandardScaler()



Processing batches: 100%|██████████| 41/41 [05:07<00:00,  7.51s/it]


In [19]:
# features = np.hstack([hog_features])
normalized_features = scaler.fit_transform(hog_features)

#save the features
# np.save('hog_features.npy', hog_features)
# np.save('sift_features.npy', sift_features_padded)
np.save('normalized_features.npy', normalized_features)

del x_data , hog_features

In [20]:
from sklearn.model_selection import train_test_split
x_train = np.load('normalized_features.npy')
# First split the data into a training set and a temporary set using a 70-30 split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_data, test_size=0.3, random_state=22, stratify=y_data)
x_images_tarin ,x_images_val, y_images_train, y_images_val = train_test_split(x_images, y_data, test_size=0.3, random_state=22, stratify=y_data)
# Then split the temporary set into a validation set and a test set using a 50-50 split


print("shape of x_train:", x_train.shape, "shape of y_train:", y_train.shape)
print("shape of x_val:", x_val.shape, "shape of y_val:", y_val.shape)

# del normalized_features, y_data

shape of x_train: (2800, 18480) shape of y_train: (2800,)
shape of x_val: (1200, 18480) shape of y_val: (1200,)


In [21]:
from sklearn.decomposition import PCA

# Create a PCA object
pca = PCA(n_components=0.9999)  # Adjust this to the desired number of components or variance to retain
# Fit the PCA model and transform the data
features_pca = pca.fit(x_train)
features_pca = pca.transform(x_train)

del x_train


In [22]:
# Encode the labels to integers
np.save('features_pca.npy', features_pca)
le = LabelEncoder()
y_data_encoded = le.fit_transform(y_train[:len(features_pca)])  # Ensure the number of labels matches the number of features

# Create an SVM classifier with gamma='scale' and C=1
clf = svm.SVC(gamma='scale', C=6.4)

# Fit the classifier on the data
clf.fit(features_pca, y_data_encoded)

del features_pca, y_data_encoded

# Now the SVM model is trained and ready to make predictions

In [23]:
from sklearn.metrics import accuracy_score# Apply PCA
features_pca = pca.transform(x_val)

# Encode the validation labels
y_val_encoded = le.transform(y_val)

# Use the trained SVM model to make predictions on the validation set

y_pred = clf.predict(features_pca)

# Compute the accuracy of the predictions
accuracy = accuracy_score(y_val_encoded, y_pred)

print(f"Validation accuracy: {accuracy * 100:.2f}%")

Validation accuracy: 95.50%


In [24]:
features_pca.shape
print(f'y_val: {y_val}')
print(f'y_pred: {y_pred}')
print(f'y_val_enc: {y_val_encoded}') 

y_val: ['0' '3' '0' ... '2' '2' '0']
y_pred: [0 3 0 ... 2 2 0]
y_val_enc: [0 3 0 ... 2 2 0]


In [25]:
from skimage.io import imsave
import os

# Create a directory to save the images
if not os.path.exists('incorrect_predictions'):
    os.makedirs('incorrect_predictions')
else:
    # If the directory already exists, delete its contents
    files = glob.glob('incorrect_predictions/*')
    for f in files:
        os.remove(f)

# Iterate over the predictions and actual labels
for i in range(len(y_pred)):
    # If the prediction is incorrect
    if y_pred[i] != y_val_encoded[i]:
        # Save the corresponding image
        imsave(f'incorrect_predictions/img_{i}.png', x_images_val[i])

NameError: name 'glob' is not defined